# 데이터 불러오기

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, KFold
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from lightgbm import LGBMRegressor
import lightgbm as lgb
import optuna
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV, ShapleyImportanceEvaluator
from optuna.integration.lightgbm import LightGBMTunerCV #lgbm 전용 tunner 가 존재
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import klib
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
X_train = pd.read_csv('X_train_1124해협지.csv')
y_train = pd.read_csv('y_train.csv').drop(columns=['ID'])
X_test = pd.read_csv('X_test_1124해협지.csv')

## 피처추가

### 1) 직무태그 글자수 생성

In [3]:
X_train = X_train.fillna('')
X_test = X_test.fillna('')

In [4]:
a = []
for i in X_train['직무태그']:
        a.append(len(i))
        
X_train['직무태그글자수'] = a

In [5]:
b = []
for j in X_test['직무태그']:
        b.append(len(j))
        
X_test['직무태그글자수'] = b

In [6]:
a = []
for i in X_train['직무태그글자수']:
    if 10 > i >= 0:
        a.append(10)
    elif 20 > i >= 10:
        a.append(20)
    elif 30 > i >= 20:
        a.append(30)
    elif 40 > i >= 30:
        a.append(40)
    elif 50 > i >= 40:
        a.append(50)
    elif 60 > i >= 50:
        a.append(60)
    elif 70 > i >= 60:
        a.append(70)
    else:
        a.append(80)

X_train['직무태그글자수범위별'] = a
#X_train = X_train.drop(columns = '직무태그글자수')

b = []
for i in X_test['직무태그글자수']:
    if 10 > i >= 0:
        b.append(10)
    elif 20 > i >= 10:
        b.append(20)
    elif 30 > i >= 20:
        b.append(30)
    elif 40 > i >= 30:
        b.append(40)
    elif 50 > i >= 40:
        b.append(50)
    elif 60 > i >= 50:
        b.append(60)
    elif 70 > i >= 60:
        b.append(70)
    else:
        b.append(80)

X_test['직무태그글자수범위별'] = b
#X_test = X_test.drop(columns = '직무태그글자수')

### 2) 근무지역 소분류, 대분류 추가

In [7]:
X_train['근무지역_소분류'] = X_train['근무지역'].copy()
X_test['근무지역_소분류'] = X_test['근무지역'].copy()

In [8]:
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.contains
                ('해외|미국|중국|러시아|일본|캐나다|필리핀|인도|싱가포르|대만|말레이시아|프랑스|홍콩')] = '해외'

X_train['근무지역_소분류'][X_train['근무지역_소분류'] == '전국,,'] = '전국'
X_train['근무지역_소분류'][X_train['근무지역_소분류'] == '전국,전국,'] = '전국'
X_train['근무지역_소분류'][X_train['근무지역_소분류'] == '전국,전국,전국'] = '전국'

X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '서울'] = '서울'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '경기'] = '경기'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '인천'] = '인천'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '부산'] = '부산'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '대전'] = '대전'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '광주'] = '광주'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '충남'] = '충남'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '경남'] = '경남'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '충북'] = '충북'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '제주'] = '제주'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '경북'] = '경북'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '강원'] = '강원'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '울산'] = '울산'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '전남'] = '전남'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '대구'] = '대구'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'

X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.contains
                ('기타')] = '기타'

In [9]:
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.contains
                ('해외|미국|중국|러시아|일본|캐나다|필리핀|인도|싱가포르|대만|말레이시아|프랑스|홍콩')] = '해외'

X_test['근무지역_소분류'][X_test['근무지역_소분류'] == '전국,,'] = '전국'
X_test['근무지역_소분류'][X_test['근무지역_소분류'] == '전국,전국,'] = '전국'
X_test['근무지역_소분류'][X_test['근무지역_소분류'] == '전국,전국,전국'] = '전국'

X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '서울'] = '서울'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '경기'] = '경기'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '인천'] = '인천'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '부산'] = '부산'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '대전'] = '대전'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '광주'] = '광주'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '충남'] = '충남'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '경남'] = '경남'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '충북'] = '충북'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '제주'] = '제주'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '경북'] = '경북'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '강원'] = '강원'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '울산'] = '울산'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '전남'] = '전남'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '대구'] = '대구'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'

X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.contains
                ('기타')] = '기타'

In [10]:
X_train['근무지역_대분류'] = X_train['근무지역_소분류'].copy()
X_test['근무지역_대분류'] = X_test['근무지역_소분류'].copy()

In [11]:
X_train['근무지역_대분류'][X_train['근무지역_대분류'].str.contains('서울|인천|경기')] = '수도권'
X_train['근무지역_대분류'][~X_train['근무지역_대분류'].str.contains('수도권|해외|전국')] = '비수도권'

X_test['근무지역_대분류'][X_test['근무지역_대분류'].str.contains('서울|인천|경기')] = '수도권'
X_test['근무지역_대분류'][~X_test['근무지역_대분류'].str.contains('수도권|해외|전국')] = '비수도권'

## 3) 근무경력 유무

In [12]:
X_train['경력유무'] = X_train['근무경력'].apply(lambda x: 'X' if (-1<x<24) else 'O')
X_test['경력유무'] = X_test['근무경력'].apply(lambda x: 'X' if (-1<x<24) else "O")

## 4) 인서울여부

In [13]:
in_seoul_lst = ['성균관대학교', '중앙대학교', '세종대학교', '연세대학교', '이화여자대학교']

X_train['인서울여부'] = X_train['출신대학'].map(lambda x: '0' if x in in_seoul_lst else '1')

X_test['인서울여부'] = X_test['출신대학'].map(lambda x: '0' if x in in_seoul_lst else '1')

## 5) 직무태그_직무수

In [14]:
X_train['직무태그_직무수']= X_train['직무태그'].apply(lambda x: len(x.split(',')))
X_test['직무태그_직무수']= X_test['직무태그'].apply(lambda x: len(x.split(',')))

## 6) 직종_직종수

In [15]:
X_train['직종수']= X_train['직종'].apply(lambda x: len(x.split('·')))
X_test['직종수']= X_test['직종'].apply(lambda x: len(x.split('·')))

## 7) 세부직종_세부직종수

In [16]:
X_train['세부직종수']= X_train['직종'].apply(lambda x: len(x.split('·')))
X_test['세부직종수']= X_test['직종'].apply(lambda x: len(x.split('·')))

## 8) 근무경력X대학성적

In [17]:
X_train['근무경력_x_대학성적'] = X_train['근무경력'] * X_train['대학성적']
X_test['근무경력_x_대학성적'] = X_test['근무경력'] * X_test['대학성적']

## 9) 대학교X대학성적

In [18]:
top_rank = ['성균관대학교', '연세대학교', '중앙대학교', '이화여자대학교']
mid_rank = ['세종대학교', '성신여자대학교']
low_rank = ['한성대학교', '동덕여자대학교', '서울여자대학교', '서울과학기술대학교']
province = ['제주대학교', '인천대학교', '군산대학교', '한국산업기술대학교']
etc= ['수원대학교', '전주대학교', '세명대학교', '신라대학교', '상지대학교', '한밭대학교', '경기대학교', '선문대학교', '성공회대학교', '호원대학교', '한림대학교', '목포대학교', '동아대학교', '동의대학교', '성결대학교', '호남대학교', '광주대학교', '서원대학교', '동서대학교', '한일장신대학교', '한세대학교', '용인대학교', '경주대학교', '협성대학교', '서울신학대학교', '송원대학교', '부산디지털대학교','남부대학교','기타']


university = []
for i in X_train['출신대학']:
    if i in top_rank:
        university.append(5)
    elif i in mid_rank:
        university.append(4)
    elif i in low_rank:
        university.append(3)
    elif i in province:
        university.append(2)
    elif i in etc:
        university.append(1)
        
X_train['출신대학수치'] = university

university = []
for i in X_test['출신대학']:
    if i in top_rank:
        university.append(5)
    elif i in mid_rank:
        university.append(4)
    elif i in low_rank:
        university.append(3)
    elif i in province:
        university.append(2)
    elif i in etc:
        university.append(1) 
        
X_test['출신대학수치'] = university

a = X_train['출신대학수치'] * X_train['대학성적']
X_train['출신대학*대학성적'] = a
a = X_test['출신대학수치'] * X_test['대학성적']
X_test['출신대학*대학성적'] = a

## 10) 세부직종 blank 없애기

In [19]:
#blank 없애기 + 세부직종 전처리
X_train['세부직종'] = X_train['세부직종'].replace('기타 기술·과학·산업','기타기술·과학·산업')
X_train['세부직종'] = X_train['세부직종'].replace('기타 경영·기획·회계·사무','기타경영·기획·회계·사무')
X_train['세부직종'] = X_train['세부직종'].replace('기타 전문·교육·자격','기타전문·교육·자격')
X_train['세부직종'] = X_train['세부직종'].replace('기타 통신·모바일','기타통신·모바일')
X_train['세부직종'] = X_train['세부직종'].replace('기타 디자인','기타디자인')
X_train['세부직종'] = X_train['세부직종'].replace('기타 건설·기계·전기·전자','기타건설·기계·전기·전자')
X_train['세부직종'] = X_train['세부직종'].replace('기타 영업·판매·TM','기타영업·판매·TM')
X_train['세부직종'] = X_train['세부직종'].replace('기타 재료·화학·섬유·의복','기타재료·화학·섬유·의복')
X_train['세부직종'] = X_train['세부직종'].replace('기타 IT·게임','기타IT·게임')
X_train['세부직종'] = X_train['세부직종'].replace('데이터베이스 전문직','데이터베이스전문직')

X_test['세부직종'] = X_test['세부직종'].replace('기타 기술·과학·산업','기타기술·과학·산업')
X_test['세부직종'] = X_test['세부직종'].replace('기타 경영·기획·회계·사무','기타경영·기획·회계·사무')
X_test['세부직종'] = X_test['세부직종'].replace('기타 전문·교육·자격','기타전문·교육·자격')
X_test['세부직종'] = X_test['세부직종'].replace('기타 통신·모바일','기타통신·모바일')
X_test['세부직종'] = X_test['세부직종'].replace('기타 디자인','기타디자인')
X_test['세부직종'] = X_test['세부직종'].replace('기타 건설·기계·전기·전자','기타건설·기계·전기·전자')
X_test['세부직종'] = X_test['세부직종'].replace('기타 영업·판매·TM','기타영업·판매·TM')
X_test['세부직종'] = X_test['세부직종'].replace('기타 재료·화학·섬유·의복','기타재료·화학·섬유·의복')
X_test['세부직종'] = X_test['세부직종'].replace('기타 IT·게임','기타IT·게임')
X_test['세부직종'] = X_test['세부직종'].replace('데이터베이스 전문직','데이터베이스전문직')

X_train['세부직종'] = X_train['세부직종'].replace('게임기획','게임디자인')
X_train['세부직종'] = X_train['세부직종'].replace('게임운영·테스터','게임프로그래머')
X_train['세부직종'] = X_train['세부직종'].replace('기타영업·판매·TM','텔레마케팅(인/아웃바운드TM)')
X_train['세부직종'] = X_train['세부직종'].replace('텔레마케팅(인/아웃바운드TM)','텔레마케팅')
X_test['세부직종'] = X_test['세부직종'].replace('텔레마케팅(인/아웃바운드TM)','텔레마케팅')
X_train['세부직종'] = X_train['세부직종'].replace('농림어업','기타')
X_train['세부직종'] = X_train['세부직종'].replace('모바일디자인','모바일프로그래머')
X_train['세부직종'] = X_train['세부직종'].replace('보험·연금','보험영업')
X_train['세부직종'] = X_train['세부직종'].replace('유비쿼터스·와이브로·WCDMA','기타')
X_train['세부직종'] = X_train['세부직종'].replace('정비·검사·수리·AS','컴퓨터·PC수리')
X_train['세부직종'] = X_train['세부직종'].replace('조리·제빵·제과','식품가공')

## 11) 세부직종별_필수직무유무

In [20]:
# 키워드의 수를 세어 딕셔내리 형태로 넣어주는 함수 
def get_counts(seq): 
    counts = {}
    for x in seq:
        if x in counts:
            counts[x] += 1
        else:
            counts[x] = 1
    return counts

In [21]:
세부직종_lst = list(X_train['세부직종'].unique())

In [22]:
세부직종_lst

['영상·음향·사진·카메라',
 '사무·총무·법무',
 '하드웨어설계·개발·관리',
 '기술영업',
 '기타기술·과학·산업',
 '응용프로그래머(Unix,Linux,Java)',
 '경영·기획·전략',
 '해외영업',
 '경리·회계',
 '마케팅·마케팅기획',
 '웹디자인',
 '영업·영업기획',
 '비서·인포메이션·수행·안내',
 '에너지·환경',
 '호텔·숙박·여행',
 '서버·네트워크·보안',
 '시스템분석·설계·컨설턴트·ERP',
 '영업관리·영업지원',
 '전기·전자·제어',
 '무역·수출입·운송사무',
 '웹프로그래머',
 '외국어·번역·통역',
 '인사·노무·교육·인재개발',
 '광고·홍보',
 '건설·토목·건축설비',
 '텔레마케팅',
 '기계·기계설계·기계설비',
 '물류·유통관리',
 '섬유·의복',
 '교육·교사·교수·강사',
 '의류·패션·잡화디자인',
 '구매·자재사무',
 '생산관리·품질관리·품질보증',
 '웹마스터',
 '식품가공',
 '법률·법무·특허·상표',
 '웹기획·웹마케팅·PM',
 '제약·의약',
 'CAD·CAM',
 '기타경영·기획·회계·사무',
 '제품·산업디자인',
 '그래픽디자인·CG',
 '신문·방송·잡지기자',
 '판매·매장관리·캐셔·프런트',
 '연구개발·R&D',
 '경영분석·컨설팅·M&A',
 '기타전문·교육·자격',
 '출판·편집디자인',
 '전산사무·전산관리',
 '기타통신·모바일',
 '금융·증권·투자',
 '네트워크·LAN',
 '동영상·멀티미디어(영상·음향)',
 '금속·금형·요업·재료·열처리',
 '생명공학·유전공학',
 '부동산·경매·감정평가',
 '화학·화공',
 '기타',
 '운영·Helpdesk·컨텐츠관리',
 '컴퓨터·IT·디자인강사',
 '정보검색·정보제공',
 '제조·생산 단순노무',
 '경비·보안·건물관리·미화',
 '캐릭터·애니메이션',
 '상담·고객지원',
 '시스템프로그래머',
 '데이터베이스전문직',
 '모바일프로그래머',
 '기타디자인',
 '운전·운송·택배·배송',
 '

In [23]:
for i in 세부직종_lst:
    globals()[f'{i}_sum'] = X_train[X_train['세부직종'] == i]['직무태그'].sum().split(',')
    globals()[f'{i}_count'] = get_counts(globals()[f'{i}_sum'])
    globals()[f'{i}_count'] = sorted(globals()[f'{i}_count'].items(), key=lambda x: x[1], reverse=True)
    
for i in 세부직종_lst:
    globals()[f'{i}_essential'] = []
    for j in globals()[f'{i}_count'][:5]:
        globals()[f'{i}_essential'].append(j[0])

In [24]:
essential = []

def essentials(x):
    tag = x['세부직종']
    try : 
        if len(set(x['직무태그'].split(',')) & set(globals()[f'{tag}_essential'])) != 0:
            essential.append(1)
        else:
            essential.append(0)
    except KeyError:
        essential.append(0)
        

X_train.apply(essentials,axis=1)
X_train['세부직종별_필수직무유무'] = essential

essential = []
X_test.apply(essentials,axis=1)
X_test['세부직종별_필수직무유무'] = essential

## 12) 숙련도

In [25]:
X_train['숙련도'] = X_train['근무경력'].apply(lambda x:'경력없음' 
                          if x==0 else '하' if x<=3 else '중' if x<=6 else '상')

X_test['숙련도'] = X_test['근무경력'].apply(lambda x:'경력없음' 
                          if x==0 else '하' if x<=3 else '중' if x<=6 else '상')

## 13) 직무태그_직무수 + 근무경력

In [26]:
X_train['직무태그_직무수 + 근무경력_x']=X_train['직무태그_직무수']*X_train['근무경력']
X_test['직무태그_직무수 + 근무경력_x']=X_test['직무태그_직무수']*X_test['근무경력']

## 14) 세부직종별 근무경력, 직무태그_직무수

In [27]:
X_train=pd.merge(X_train, X_train.groupby('세부직종')['근무경력'].mean().reset_index().rename(columns={'근무경력':'세부직종별 근무경력mean'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['근무경력'].mean().reset_index().rename(columns={'근무경력':'세부직종별 근무경력mean'}), on = '세부직종',how='left')

X_train=pd.merge(X_train, X_train.groupby('세부직종')['근무경력'].median().reset_index().rename(columns={'근무경력':'세부직종별 근무경력median'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['근무경력'].median().reset_index().rename(columns={'근무경력':'세부직종별 근무경력median'}), on = '세부직종',how='left')

X_train=pd.merge(X_train,X_train.groupby('세부직종')['근무경력'].var().reset_index().rename(columns={'근무경력':'세부직종별 근무경력var'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['근무경력'].var().reset_index().rename(columns={'근무경력':'세부직종별 근무경력var'}), on = '세부직종',how='left')

X_train=pd.merge(X_train, X_train.groupby('세부직종')['근무경력'].std().reset_index().rename(columns={'근무경력':'세부직종별 근무경력std'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['근무경력'].std().reset_index().rename(columns={'근무경력':'세부직종별 근무경력std'}), on = '세부직종',how='left')

X_train=pd.merge(X_train, X_train.groupby('세부직종')['근무경력'].sum().reset_index().rename(columns={'근무경력':'세부직종별 근무경력sum'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['근무경력'].sum().reset_index().rename(columns={'근무경력':'세부직종별 근무경력sum'}), on = '세부직종',how='left')

X_train=pd.merge(X_train, X_train.groupby('세부직종')['근무경력'].min().reset_index().rename(columns={'근무경력':'세부직종별 근무경력min'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['근무경력'].min().reset_index().rename(columns={'근무경력':'세부직종별 근무경력min'}), on = '세부직종',how='left')

X_train=pd.merge(X_train, X_train.groupby('세부직종')['근무경력'].max().reset_index().rename(columns={'근무경력':'세부직종별 근무경력max'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['근무경력'].max().reset_index().rename(columns={'근무경력':'세부직종별 근무경력max'}), on = '세부직종',how='left')

## 15) 세부직종별 직무태그_직무수

In [28]:
X_train=pd.merge(X_train, X_train.groupby('세부직종')['직무태그_직무수'].mean().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수mean'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['직무태그_직무수'].mean().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수mean'}), on = '세부직종',how='left')

X_train=pd.merge(X_train, X_train.groupby('세부직종')['직무태그_직무수'].median().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수median'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['직무태그_직무수'].median().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수median'}), on = '세부직종',how='left')

X_train=pd.merge(X_train, X_train.groupby('세부직종')['직무태그_직무수'].var().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수var'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['직무태그_직무수'].var().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수var'}), on = '세부직종',how='left')

X_train=pd.merge(X_train, X_train.groupby('세부직종')['직무태그_직무수'].std().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수std'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['직무태그_직무수'].std().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수std'}), on = '세부직종',how='left')

X_train=pd.merge(X_train, X_train.groupby('세부직종')['직무태그_직무수'].sum().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수sum'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['직무태그_직무수'].sum().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수sum'}), on = '세부직종',how='left')

X_train=pd.merge(X_train, X_train.groupby('세부직종')['직무태그_직무수'].min().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수min'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['직무태그_직무수'].max().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수min'}), on = '세부직종',how='left')

X_train=pd.merge(X_train, X_train.groupby('세부직종')['직무태그_직무수'].max().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수max'}), on = '세부직종',how='left')
X_test=pd.merge(X_test, X_test.groupby('세부직종')['직무태그_직무수'].max().reset_index().rename(columns={'직무태그_직무수':'세부직종별 직무태그_직무수max'}), on = '세부직종',how='left')

## 16) 출신대학별 근무경력,대학성적

In [29]:
X_train=pd.merge(X_train, X_train.groupby('출신대학')['근무경력'].mean().reset_index().rename(columns={'근무경력':'출신대학별 근무경력mean'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['근무경력'].mean().reset_index().rename(columns={'근무경력':'출신대학별 근무경력mean'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['근무경력'].sum().reset_index().rename(columns={'근무경력':'출신대학별 근무경력sum'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['근무경력'].sum().reset_index().rename(columns={'근무경력':'출신대학별 근무경력sum'}), on = '출신대학',how='left')


X_train=pd.merge(X_train, X_train.groupby('출신대학')['근무경력'].median().reset_index().rename(columns={'근무경력':'출신대학별 근무경력median'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['근무경력'].median().reset_index().rename(columns={'근무경력':'출신대학별 근무경력median'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['근무경력'].var().reset_index().rename(columns={'근무경력':'출신대학별 근무경력var'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['근무경력'].var().reset_index().rename(columns={'근무경력':'출신대학별 근무경력var'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['근무경력'].std().reset_index().rename(columns={'근무경력':'출신대학별 근무경력std'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['근무경력'].std().reset_index().rename(columns={'근무경력':'출신대학별 근무경력std'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['근무경력'].min().reset_index().rename(columns={'근무경력':'출신대학별 근무경력min'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['근무경력'].min().reset_index().rename(columns={'근무경력':'출신대학별 근무경력min'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['근무경력'].max().reset_index().rename(columns={'근무경력':'출신대학별 근무경력max'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['근무경력'].max().reset_index().rename(columns={'근무경력':'출신대학별 근무경력max'}), on = '출신대학',how='left')

In [30]:
X_train=pd.merge(X_train, X_train.groupby('출신대학')['대학성적'].mean().reset_index().rename(columns={'대학성적':'출신대학별 대학성적mean'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['대학성적'].mean().reset_index().rename(columns={'대학성적':'출신대학별 대학성적mean'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['대학성적'].median().reset_index().rename(columns={'대학성적':'출신대학별 대학성적median'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['대학성적'].median().reset_index().rename(columns={'대학성적':'출신대학별 대학성적median'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['대학성적'].var().reset_index().rename(columns={'대학성적':'출신대학별 대학성적var'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['대학성적'].var().reset_index().rename(columns={'대학성적':'출신대학별 대학성적var'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['대학성적'].std().reset_index().rename(columns={'대학성적':'출신대학별 대학성적std'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['대학성적'].std().reset_index().rename(columns={'대학성적':'출신대학별 대학성적std'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['대학성적'].sum().reset_index().rename(columns={'대학성적':'출신대학별 대학성적sum'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['대학성적'].sum().reset_index().rename(columns={'대학성적':'출신대학별 대학성적sum'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['대학성적'].min().reset_index().rename(columns={'대학성적':'출신대학별 대학성적min'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['대학성적'].min().reset_index().rename(columns={'대학성적':'출신대학별 대학성적min'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['대학성적'].max().reset_index().rename(columns={'대학성적':'출신대학별 대학성적max'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['대학성적'].max().reset_index().rename(columns={'대학성적':'출신대학별 대학성적max'}), on = '출신대학',how='left')

## 17) 근무형태별 근무경력,대학성적,출신대학수치

In [31]:
X_train=pd.merge(X_train, X_train.groupby('근무형태')['근무경력'].mean().reset_index().rename(columns={'근무경력':'근무형태별 근무경력mean'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['근무경력'].mean().reset_index().rename(columns={'근무경력':'근무형태별 근무경력mean'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['근무경력'].median().reset_index().rename(columns={'근무경력':'근무형태별 근무경력median'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['근무경력'].median().reset_index().rename(columns={'근무경력':'근무형태별 근무경력median'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['근무경력'].var().reset_index().rename(columns={'근무경력':'근무형태별 근무경력var'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['근무경력'].var().reset_index().rename(columns={'근무경력':'근무형태별 근무경력var'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['근무경력'].std().reset_index().rename(columns={'근무경력':'근무형태별 근무경력std'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['근무경력'].std().reset_index().rename(columns={'근무경력':'근무형태별 근무경력std'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['근무경력'].sum().reset_index().rename(columns={'근무경력':'근무형태별 근무경력sum'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['근무경력'].sum().reset_index().rename(columns={'근무경력':'근무형태별 근무경력sum'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['근무경력'].min().reset_index().rename(columns={'근무경력':'근무형태별 근무경력min'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['근무경력'].min().reset_index().rename(columns={'근무경력':'근무형태별 근무경력min'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['근무경력'].max().reset_index().rename(columns={'근무경력':'근무형태별 근무경력max'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['근무경력'].max().reset_index().rename(columns={'근무경력':'근무형태별 근무경력max'}), on = '근무형태',how='left')


In [32]:
X_train=pd.merge(X_train, X_train.groupby('근무형태')['대학성적'].mean().reset_index().rename(columns={'대학성적':'근무형태별 대학성적mean'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['대학성적'].mean().reset_index().rename(columns={'대학성적':'근무형태별 대학성적mean'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['대학성적'].median().reset_index().rename(columns={'대학성적':'근무형태별 대학성적median'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['대학성적'].median().reset_index().rename(columns={'대학성적':'근무형태별 대학성적median'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['대학성적'].var().reset_index().rename(columns={'대학성적':'근무형태별 대학성적var'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['대학성적'].var().reset_index().rename(columns={'대학성적':'근무형태별 대학성적var'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['대학성적'].std().reset_index().rename(columns={'대학성적':'근무형태별 대학성적std'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['대학성적'].std().reset_index().rename(columns={'대학성적':'근무형태별 대학성적std'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['대학성적'].sum().reset_index().rename(columns={'대학성적':'근무형태별 대학성적sum'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['대학성적'].sum().reset_index().rename(columns={'대학성적':'근무형태별 대학성적sum'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['대학성적'].min().reset_index().rename(columns={'대학성적':'근무형태별 대학성적min'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['대학성적'].min().reset_index().rename(columns={'대학성적':'근무형태별 대학성적min'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['대학성적'].max().reset_index().rename(columns={'대학성적':'근무형태별 대학성적max'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['대학성적'].max().reset_index().rename(columns={'대학성적':'근무형태별 대학성적max'}), on = '근무형태',how='left')

In [33]:
X_train=pd.merge(X_train, X_train.groupby('근무형태')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치mean'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치mean'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치median'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치median'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치sum'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치sum'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치var'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치var'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치std'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치std'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치min'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치min'}), on = '근무형태',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무형태')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치max'}), on = '근무형태',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무형태')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'근무형태별 출신대학수치max'}), on = '근무형태',how='left')

## 18) 대학전공별 근무경력, 대학성적, 출신대학수치

In [34]:
X_train=pd.merge(X_train, X_train.groupby('대학전공')['근무경력'].mean().reset_index().rename(columns={'근무경력':'대학전공별 근무경력mean'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['근무경력'].mean().reset_index().rename(columns={'근무경력':'대학전공별 근무경력mean'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['근무경력'].median().reset_index().rename(columns={'근무경력':'대학전공별 근무경력median'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['근무경력'].median().reset_index().rename(columns={'근무경력':'대학전공별 근무경력median'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['근무경력'].var().reset_index().rename(columns={'근무경력':'대학전공별 근무경력var'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['근무경력'].var().reset_index().rename(columns={'근무경력':'대학전공별 근무경력var'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['근무경력'].std().reset_index().rename(columns={'근무경력':'대학전공별 근무경력std'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['근무경력'].std().reset_index().rename(columns={'근무경력':'대학전공별 근무경력std'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['근무경력'].sum().reset_index().rename(columns={'근무경력':'대학전공별 근무경력sum'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['근무경력'].sum().reset_index().rename(columns={'근무경력':'대학전공별 근무경력sum'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['근무경력'].min().reset_index().rename(columns={'근무경력':'대학전공별 근무경력min'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['근무경력'].min().reset_index().rename(columns={'근무경력':'대학전공별 근무경력min'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['근무경력'].max().reset_index().rename(columns={'근무경력':'대학전공별 근무경력max'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['근무경력'].max().reset_index().rename(columns={'근무경력':'대학전공별 근무경력max'}), on = '대학전공',how='left')

In [35]:
X_train=pd.merge(X_train, X_train.groupby('대학전공')['대학성적'].mean().reset_index().rename(columns={'대학성적':'대학전공별 대학성적mean'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['대학성적'].mean().reset_index().rename(columns={'대학성적':'대학전공별 대학성적mean'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['대학성적'].median().reset_index().rename(columns={'대학성적':'대학전공별 대학성적median'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['대학성적'].median().reset_index().rename(columns={'대학성적':'대학전공별 대학성적median'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['대학성적'].var().reset_index().rename(columns={'대학성적':'대학전공별 대학성적var'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['대학성적'].var().reset_index().rename(columns={'대학성적':'대학전공별 대학성적var'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['대학성적'].std().reset_index().rename(columns={'대학성적':'대학전공별 대학성적std'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['대학성적'].std().reset_index().rename(columns={'대학성적':'대학전공별 대학성적std'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['대학성적'].sum().reset_index().rename(columns={'대학성적':'대학전공별 대학성적sum'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['대학성적'].sum().reset_index().rename(columns={'대학성적':'대학전공별 대학성적sum'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['대학성적'].min().reset_index().rename(columns={'대학성적':'대학전공별 대학성적min'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['대학성적'].min().reset_index().rename(columns={'대학성적':'대학전공별 대학성적min'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['대학성적'].max().reset_index().rename(columns={'대학성적':'대학전공별 대학성적max'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['대학성적'].max().reset_index().rename(columns={'대학성적':'대학전공별 대학성적max'}), on = '대학전공',how='left')

In [36]:
X_train=pd.merge(X_train, X_train.groupby('대학전공')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치mean'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치mean'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치median'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치median'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치sum'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치sum'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치var'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치var'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치std'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치std'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치min'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치min'}), on = '대학전공',how='left')

X_train=pd.merge(X_train, X_train.groupby('대학전공')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치max'}), on = '대학전공',how='left')
X_test=pd.merge(X_test, X_test.groupby('대학전공')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'대학전공별 출신대학수치max'}), on = '대학전공',how='left')

## 19) 직무태그별 근무경력, 대학성적, 출신대학수치

In [37]:
X_train=pd.merge(X_train, X_train.groupby('직무태그')['근무경력'].mean().reset_index().rename(columns={'근무경력':'직무태그별 근무경력mean'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['근무경력'].mean().reset_index().rename(columns={'근무경력':'직무태그별 근무경력mean'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['근무경력'].median().reset_index().rename(columns={'근무경력':'직무태그별 근무경력median'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['근무경력'].median().reset_index().rename(columns={'근무경력':'직무태그별 근무경력median'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['근무경력'].var().reset_index().rename(columns={'근무경력':'직무태그별 근무경력var'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['근무경력'].var().reset_index().rename(columns={'근무경력':'직무태그별 근무경력var'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['근무경력'].std().reset_index().rename(columns={'근무경력':'직무태그별 근무경력std'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['근무경력'].std().reset_index().rename(columns={'근무경력':'직무태그별 근무경력std'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['근무경력'].sum().reset_index().rename(columns={'근무경력':'직무태그별 근무경력sum'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['근무경력'].sum().reset_index().rename(columns={'근무경력':'직무태그별 근무경력sum'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['근무경력'].min().reset_index().rename(columns={'근무경력':'직무태그별 근무경력min'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['근무경력'].min().reset_index().rename(columns={'근무경력':'직무태그별 근무경력min'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['근무경력'].max().reset_index().rename(columns={'근무경력':'직무태그별 근무경력max'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['근무경력'].max().reset_index().rename(columns={'근무경력':'직무태그별 근무경력max'}), on = '직무태그',how='left')

In [38]:
X_train=pd.merge(X_train, X_train.groupby('직무태그')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치mean'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치mean'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치median'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치median'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치sum'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치sum'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치var'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치var'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치std'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치std'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치min'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치min'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치max'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'직무태그별 출신대학수치max'}), on = '직무태그',how='left')

## 20) 출신대학+전공

In [39]:
X_train['대학+전공'] = X_train['출신대학'] + X_train['대학전공']
X_test['대학+전공'] = X_test['출신대학'] + X_test['대학전공']

## 21) 근무형태 + 직종

In [40]:
X_train['근무형태+직종'] = X_train['근무형태']+X_train['직종']
X_test['근무형태+직종'] = X_test['근무형태']+X_test['직종']

## 22) 세부직종 + 출신대학

In [41]:
X_train['세부직종+출신대학'] = X_train['세부직종']+X_train['출신대학']
X_test['세부직종+출신대학'] = X_test['세부직종']+X_test['출신대학']

## 23) 세부직종 + 대학전공

In [42]:
X_train['세부직종+대학전공'] = X_train['세부직종']+X_train['대학전공']
X_test['세부직종+대학전공'] = X_test['세부직종']+X_test['대학전공']

## 24) 출신대학 + 대학전공

In [43]:
X_train['대학전공+출신대학'] = X_train['대학전공']+X_train['출신대학']
X_test['대학전공+출신대학'] = X_test['대학전공']+X_test['출신대학']

## 25) 직무태그1(맨 앞에 있는 태그만 추출)

In [44]:
a = []
for i in X_train['직무태그']:
    a.append(i.split(',')[0])
    
X_train['직무태그1']= a

In [45]:
b = []
for i in X_test['직무태그']:
    b.append(i.split(',')[0])

X_test['직무태그1']= b

## 26) 직종, 및 세부직종별 평균,중앙값, 합, 표준편차, 최솟값, 최댓값

In [46]:
X_train['직종'] = X_train['직종'].apply(lambda x:  x if x in (X_train.직종.unique()) else '기타 직종'  )

X_test['직종'] = X_test['직종'].apply(lambda x:  x if x in (X_train.직종.unique()) else '기타 직종'  )

job_score = dict(X_train.groupby('직종')['대학성적'].mean())

X_train['직종별대학성적평균'] = X_train['직종'].apply(lambda x: job_score[x])
X_test['직종별대학성적평균'] = X_test['직종'].apply(lambda x: job_score[x])

job_score = dict(X_train.groupby('직종')['대학성적'].median())

X_train['직종별대학성적최빈값'] = X_train['직종'].apply(lambda x: job_score[x])
X_test['직종별대학성적최빈값'] = X_test['직종'].apply(lambda x: job_score[x])

job_score = X_train.groupby('직종')['대학성적'].sum()

X_train['직종별대학성적합'] = X_train['직종'].apply(lambda x: job_score[x])
X_test['직종별대학성적합'] = X_test['직종'].apply(lambda x: job_score[x])

job_score = X_train.groupby('직종')['대학성적'].std().fillna(0)

X_train['직종별대학성적표준편차'] = X_train['직종'].apply(lambda x: job_score[x])
X_test['직종별대학성적표준편차'] = X_test['직종'].apply(lambda x: job_score[x])

job_score = X_train.groupby('직종')['대학성적'].min()

X_train['직종별대학성적최소값'] = X_train['직종'].apply(lambda x: job_score[x])
X_test['직종별대학성적최소값'] = X_test['직종'].apply(lambda x: job_score[x])

job_score = X_train.groupby('직종')['대학성적'].max()

X_train['직종별대학성적최대값'] = X_train['직종'].apply(lambda x: job_score[x])
X_test['직종별대학성적최대값'] = X_test['직종'].apply(lambda x: job_score[x])


detail_job_score = X_train.groupby('직종')['근무경력'].mean()

X_train['직종별근무경력평균'] = X_train['직종'].apply(lambda x: detail_job_score[x])
X_test['직종별근무경력평균'] = X_test['직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('직종')['근무경력'].median()

X_train['직종별근무경력최빈값'] = X_train['직종'].apply(lambda x: detail_job_score[x])
X_test['직종별근무경력최빈값'] = X_test['직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('직종')['근무경력'].sum()

X_train['직종별근무경력합'] = X_train['직종'].apply(lambda x: detail_job_score[x])
X_test['직종별근무경력합'] = X_test['직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('직종')['근무경력'].std().fillna(0)

X_train['직종별근무경력표준편차'] = X_train['직종'].apply(lambda x: detail_job_score[x])
X_test['직종별근무경력표준편차'] = X_test['직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('직종')['근무경력'].min()

X_train['직종별근무경력최소값'] = X_train['직종'].apply(lambda x: detail_job_score[x])
X_test['직종별근무경력최소값'] = X_test['직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('직종')['대학성적'].max()

X_train['직종별대학성적최대값'] = X_train['직종'].apply(lambda x: detail_job_score[x])
X_test['직종별대학성적최대값'] = X_test['직종'].apply(lambda x: detail_job_score[x])

## 세부직종

X_train.세부직종.nunique()

X_test.세부직종.nunique()

X_train['세부직종'] = X_train['세부직종'].apply(lambda x:  x if x in (X_train.세부직종.unique()) else '기타'  )

X_test['세부직종'] = X_test['세부직종'].apply(lambda x:  x if x in (X_train.세부직종.unique()) else '기타'  )

detail_job_score = X_train.groupby('세부직종')['대학성적'].mean()

X_train['세부직종별대학성적평균'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별대학성적평균'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('세부직종')['대학성적'].median() 

X_train['세부직종별대학성적최빈값'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별대학성적최빈값'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('세부직종')['대학성적'].sum()

X_train['세부직종별대학성적합'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별대학성적합'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('세부직종')['대학성적'].std().fillna(0)

X_train['세부직종별대학성적표준편차'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별대학성적표준편차'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('세부직종')['대학성적'].min()

X_train['세부직종별대학성적최소값'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별대학성적최소값'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('세부직종')['대학성적'].max()

X_train['세부직종별대학성적최대값'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별대학성적최대값'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('세부직종')['근무경력'].mean()

X_train['세부직종별근무경력평균'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별근무경력평균'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('세부직종')['근무경력'].median()

X_train['세부직종별근무경력최빈값'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별근무경력최빈값'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('세부직종')['근무경력'].sum()

X_train['세부직종별근무경력합'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별근무경력합'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('세부직종')['근무경력'].std().fillna(0)

X_train['세부직종별근무경력표준편차'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별근무경력표준편차'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('세부직종')['근무경력'].min()

X_train['세부직종별근무경력최소값'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별근무경력최소값'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('세부직종')['근무경력'].max()

X_train['세부직종별근무경력최대값'] = X_train['세부직종'].apply(lambda x: detail_job_score[x])
X_test['세부직종별근무경력최대값'] = X_test['세부직종'].apply(lambda x: detail_job_score[x])

## 27) 수정해서 보냈삼

## 27) 어학시험별 대학성적,근무경력 평균, 중앙값, 합, 표준편차, 최소, 최대,

In [47]:
X_train['어학시험'] = X_train['어학시험'].apply(lambda x:  x if x in (X_train.어학시험.unique()) else '기타'  )

X_test['어학시험'] = X_test['어학시험'].apply(lambda x:  x if x in (X_train.어학시험.unique()) else '기타'  )

language_score = X_train.groupby('어학시험')['대학성적'].mean()

X_train['어학시험별대학성적평균'] = X_train['어학시험'].apply(lambda x:language_score[x])
X_test['어학시험별대학성적평균'] = X_test['어학시험'].apply(lambda x: language_score[x]) 

language_score = X_train.groupby('어학시험')['대학성적'].median()

X_train['어학시험별대학성적최빈값'] = X_train['어학시험'].apply(lambda x:language_score[x])
X_test['어학시험별대학성적최빈값'] = X_test['어학시험'].apply(lambda x: language_score[x])

language_score = X_train.groupby('어학시험')['대학성적'].sum()

X_train['어학시험별대학성적합'] = X_train['어학시험'].apply(lambda x:language_score[x])
X_test['어학시험별대학성적합'] = X_test['어학시험'].apply(lambda x: language_score[x])

language_score = X_train.groupby('어학시험')['대학성적'].std().fillna(0)

X_train['어학시험별대학성적표준편차'] = X_train['어학시험'].apply(lambda x:language_score[x])
X_test['어학시험별대학성적표준편차'] = X_test['어학시험'].apply(lambda x: language_score[x])

language_score = X_train.groupby('어학시험')['대학성적'].min()

X_train['어학시험별대학성적최소값'] = X_train['어학시험'].apply(lambda x:language_score[x])
X_test['어학시험별대학성적최소값'] = X_test['어학시험'].apply(lambda x: language_score[x])

language_score = X_train.groupby('어학시험')['대학성적'].max()

X_train['어학시험별대학성적최대값'] = X_train['어학시험'].apply(lambda x:language_score[x])
X_test['어학시험별대학성적최대값'] = X_test['어학시험'].apply(lambda x: language_score[x])


detail_job_score = X_train.groupby('어학시험')['근무경력'].mean()

X_train['어학시험별근무경력평균'] = X_train['어학시험'].apply(lambda x: detail_job_score[x])
X_test['어학시험별근무경력평균'] = X_test['어학시험'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('어학시험')['근무경력'].median()

X_train['어학시험별근무경력최빈값'] = X_train['어학시험'].apply(lambda x: detail_job_score[x])
X_test['어학시험별근무경력최빈값'] = X_test['어학시험'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('어학시험')['근무경력'].sum()

X_train['어학시험별근무경력합'] = X_train['어학시험'].apply(lambda x: detail_job_score[x])
X_test['어학시험별근무경력합'] = X_test['어학시험'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('어학시험')['근무경력'].std().fillna(0)

X_train['어학시험별근무경력표준편차'] = X_train['어학시험'].apply(lambda x: detail_job_score[x])
X_test['어학시험별근무경력표준편차'] = X_test['어학시험'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('어학시험')['근무경력'].min()

X_train['어학시험별근무경력최소값'] = X_train['어학시험'].apply(lambda x: detail_job_score[x])
X_test['어학시험별근무경력최소값'] = X_test['어학시험'].apply(lambda x: detail_job_score[x])

detail_job_score = X_train.groupby('어학시험')['근무경력'].max()

X_train['어학시험별근무경력최대값'] = X_train['어학시험'].apply(lambda x: detail_job_score[x])
X_test['어학시험별근무경력최대값'] = X_test['어학시험'].apply(lambda x: detail_job_score[x])

## 28) 서울근무

In [48]:
서울근무 = []
for i in X_train['근무지역'] :
    if '서울' in i :
        서울근무.append('YES')
    else :
        서울근무.append('NO')
X_train['서울근무'] = 서울근무

In [49]:
서울근무 = []
for i in X_test['근무지역'] :
    if '서울' in i :
        서울근무.append('YES')
    else :
        서울근무.append('NO')
X_test['서울근무'] = 서울근무

## 29) 피처의 제곱

In [50]:
X_train['근무경력^2'] = np.square(X_train['근무경력'])
X_test['근무경력^2'] = np.square(X_test['근무경력'])

X_train['직무태그글자수범위별^2'] = np.square(X_train['직무태그글자수범위별'])
X_test['직무태그글자수범위별^2'] = np.square(X_test['직무태그글자수범위별'])

X_train['직무태그_직무수^2'] = np.square(X_train['직무태그_직무수'])
X_test['직무태그_직무수^2'] = np.square(X_test['직무태그_직무수'])

X_train['출신대학별 근무경력sum^2'] = np.square(X_train['출신대학별 근무경력sum'])
X_test['출신대학별 근무경력sum^2'] = np.square(X_test['출신대학별 근무경력sum'])

In [51]:
X_test.shape

(11048, 162)

## 30) 출신대학*대학성적

In [52]:
top_rank = ['성균관대학교', '연세대학교', '중앙대학교', '이화여자대학교']
mid_rank = ['세종대학교', '성신여자대학교']
low_rank = ['한성대학교', '동덕여자대학교', '서울여자대학교', '서울과학기술대학교']
province = ['제주대학교', '인천대학교', '군산대학교', '한국산업기술대학교']
etc= ['수원대학교', '전주대학교', '세명대학교', '신라대학교', '상지대학교', '한밭대학교', '경기대학교', '선문대학교', '성공회대학교', '호원대학교', '한림대학교', '목포대학교', '동아대학교', '동의대학교', '성결대학교', '호남대학교', '광주대학교', '서원대학교', '동서대학교', '한일장신대학교', '한세대학교', '용인대학교', '경주대학교', '협성대학교', '서울신학대학교', '송원대학교', '부산디지털대학교','남부대학교','기타']


university = []
for i in X_train['출신대학']:
    if i in top_rank:
        university.append(5)
    elif i in mid_rank:
        university.append(4)
    elif i in low_rank:
        university.append(3)
    elif i in province:
        university.append(2)
    elif i in etc:
        university.append(1)
        
X_train['출신대학수치'] = university

university = []
for i in X_test['출신대학']:
    if i in top_rank:
        university.append(5)
    elif i in mid_rank:
        university.append(4)
    elif i in low_rank:
        university.append(3)
    elif i in province:
        university.append(2)
    elif i in etc:
        university.append(1) 
        
X_test['출신대학수치'] = university

a = X_train['출신대학수치'] * X_train['대학성적']
X_train['출신대학*대학성적'] = a
a = X_test['출신대학수치'] * X_test['대학성적']
X_test['출신대학*대학성적'] = a


## 31) 근무형태

In [53]:
X_train['근무형태_계해정'] = X_train['근무형태']
X_train['근무형태_계정해'] = X_train['근무형태']
X_train['근무형태_정계해'] = X_train['근무형태']
X_train['근무형태_정해계'] = X_train['근무형태']
X_train['근무형태_해정계'] = X_train['근무형태']
X_train['근무형태_해계정'] = X_train['근무형태']

In [54]:
X_test['근무형태_계해정'] = X_test['근무형태']
X_test['근무형태_계정해'] = X_test['근무형태']
X_test['근무형태_정계해'] = X_test['근무형태']
X_test['근무형태_정해계'] = X_test['근무형태']
X_test['근무형태_해정계'] = X_test['근무형태']
X_test['근무형태_해계정'] = X_test['근무형태']

In [55]:
#계약직(774), 해외취업(205), 정규직(15416)
X_train['근무형태_계해정'][X_train['근무형태_계해정'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'
X_train['근무형태_계해정'][X_train['근무형태_계해정'].str.contains('해외취업')] = '해외취업'
X_train['근무형태_계해정'][X_train['근무형태_계해정'].str.contains('정규직')] = '정규직'

X_test['근무형태_계해정'][X_test['근무형태_계해정'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'
X_test['근무형태_계해정'][X_test['근무형태_계해정'].str.contains('해외취업')] = '해외취업'
X_test['근무형태_계해정'][X_test['근무형태_계해정'].str.contains('정규직')] = '정규직'

In [56]:
#계약직(774), 정규직(15615), 해외취업(6)
X_train['근무형태_계정해'][X_train['근무형태_계정해'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'
X_train['근무형태_계정해'][X_train['근무형태_계정해'].str.contains('정규직')] = '정규직'
X_train['근무형태_계정해'][X_train['근무형태_계정해'].str.contains('해외취업')] = '해외취업'

X_test['근무형태_계정해'][X_test['근무형태_계정해'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'
X_test['근무형태_계정해'][X_test['근무형태_계정해'].str.contains('정규직')] = '정규직'
X_test['근무형태_계정해'][X_test['근무형태_계정해'].str.contains('해외취업')] = '해외취업'

In [57]:
#정규직(16307), 계약직(82), 해외취업(6)
X_train['근무형태_정계해'][X_train['근무형태_정계해'].str.contains('정규직')] = '정규직'
X_train['근무형태_정계해'][X_train['근무형태_정계해'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'
X_train['근무형태_정계해'][X_train['근무형태_정계해'].str.contains('해외취업')] = '해외취업'

X_test['근무형태_정계해'][X_test['근무형태_정계해'].str.contains('정규직')] = '정규직'
X_test['근무형태_정계해'][X_test['근무형태_정계해'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'
X_test['근무형태_정계해'][X_test['근무형태_정계해'].str.contains('해외취업')] = '해외취업'

In [58]:
#정규직(16307), 해외취업(12), 계약직(76)
X_train['근무형태_정해계'][X_train['근무형태_정해계'].str.contains('정규직')] = '정규직'
X_train['근무형태_정해계'][X_train['근무형태_정해계'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'
X_train['근무형태_정해계'][X_train['근무형태_정해계'].str.contains('해외취업')] = '해외취업'

X_test['근무형태_정해계'][X_test['근무형태_정해계'].str.contains('정규직')] = '정규직'
X_test['근무형태_정해계'][X_test['근무형태_정해계'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'
X_test['근무형태_정해계'][X_test['근무형태_정해계'].str.contains('해외취업')] = '해외취업'

In [59]:
#해외취업(350), 정규직(15969), 계약직(76)
X_train['근무형태_해정계'][X_train['근무형태_해정계'].str.contains('해외취업')] = '해외취업'
X_train['근무형태_해정계'][X_train['근무형태_해정계'].str.contains('정규직')] = '정규직'
X_train['근무형태_해정계'][X_train['근무형태_해정계'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'

X_test['근무형태_해정계'][X_test['근무형태_해정계'].str.contains('해외취업')] = '해외취업'
X_test['근무형태_해정계'][X_test['근무형태_해정계'].str.contains('정규직')] = '정규직'
X_test['근무형태_해정계'][X_test['근무형태_해정계'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'

In [60]:
#해외취업(350), 계약직(629), 정규직(15416)
X_train['근무형태_해계정'][X_train['근무형태_해계정'].str.contains('해외취업')] = '해외취업'
X_train['근무형태_해계정'][X_train['근무형태_해계정'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'
X_train['근무형태_해계정'][X_train['근무형태_해계정'].str.contains('정규직')] = '정규직'

X_test['근무형태_해계정'][X_test['근무형태_해계정'].str.contains('해외취업')] = '해외취업'
X_test['근무형태_해계정'][X_test['근무형태_해계정'].str.contains('계약직|인턴|파견직|병역특례')] = '계약직'
X_test['근무형태_해계정'][X_test['근무형태_해계정'].str.contains('정규직')] = '정규직'

In [61]:
X_train['근무형태_정계(해)'] = X_train['근무형태']
X_train['근무형태_계(해)정'] = X_train['근무형태']

In [62]:
X_test['근무형태_정계(해)'] = X_test['근무형태']
X_test['근무형태_계(해)정'] = X_test['근무형태']

In [63]:
#정규직(16307), 계약직(88 + 해외취업)
X_train['근무형태_정계(해)'][X_train['근무형태_정계(해)'].str.contains('정규직')] = '정규직'
X_train['근무형태_정계(해)'][X_train['근무형태_정계(해)'].str.contains('비정규직|계약직|해외취업|인턴|파견직|병역특례')] = '계약직'

X_test['근무형태_정계(해)'][X_test['근무형태_정계(해)'].str.contains('정규직')] = '정규직'
X_test['근무형태_정계(해)'][X_test['근무형태_정계(해)'].str.contains('비정규직|계약직|해외취업|인턴|파견직|병역특례')] = '계약직'

In [64]:
#계약직(10776 + 해외취업), 정규직(5619)
X_train['근무형태_계(해)정'][X_train['근무형태_계(해)정'].str.contains('비정규직|계약직|해외취업|인턴|파견직|병역특례')] = '계약직'
X_train['근무형태_계(해)정'][X_train['근무형태_계(해)정'].str.contains('정규직')] = '정규직'

X_test['근무형태_계(해)정'][X_test['근무형태_계(해)정'].str.contains('비정규직|계약직|해외취업|인턴|파견직|병역특례')] = '계약직'
X_test['근무형태_계(해)정'][X_test['근무형태_계(해)정'].str.contains('정규직')] = '정규직'

In [65]:
X_train['근무형태_정(해)계'] = X_train['근무형태']
X_train['근무형태_계정(해)'] = X_train['근무형태']

In [66]:
X_test['근무형태_정(해)계'] = X_test['근무형태']
X_test['근무형태_계정(해)'] = X_test['근무형태']

In [67]:
#정규직(16319 + 해외취업), 계약직(76)
X_train['근무형태_정(해)계'][X_train['근무형태_정(해)계'].str.contains('정규직|해외취업')] = '정규직'
X_train['근무형태_정(해)계'][X_train['근무형태_정(해)계'].str.contains('비정규직|계약직|인턴|파견직|병역특례')] = '계약직'

X_test['근무형태_정(해)계'][X_test['근무형태_정(해)계'].str.contains('정규직|해외취업')] = '정규직'
X_test['근무형태_정(해)계'][X_test['근무형태_정(해)계'].str.contains('비정규직|계약직|인턴|파견직|병역특례')] = '계약직'

In [68]:
#계약직(10571), 정규직(5824 + 해외취업)
X_train['근무형태_계정(해)'][X_train['근무형태_계정(해)'].str.contains('비정규직|계약직|인턴|파견직|병역특례')] = '계약직'
X_train['근무형태_계정(해)'][X_train['근무형태_계정(해)'].str.contains('정규직|해외취업')] = '정규직'

X_test['근무형태_계정(해)'][X_test['근무형태_계정(해)'].str.contains('비정규직|계약직|인턴|파견직|병역특례')] = '계약직'
X_test['근무형태_계정(해)'][X_test['근무형태_계정(해)'].str.contains('정규직|해외취업')] = '정규직'

## 32) 근무형태_단순화 및 베테랑정규직

In [69]:
X_train['근무형태'].fillna('N', inplace=True)
X_test['근무형태'].fillna('N', inplace=True)

In [70]:
c1 = X_train['근무형태'].str.split(',',expand=True).isnull().sum(axis=1) ==5
c2 = X_train['근무형태'].str.split(',',expand=True)[0] =='정규직'

c1_c2 = X_train.loc[(c1&c2),'근무형태'].index

X_train['근무형태_단순화'] = X_train.index.isin(c1_c2)
X_test['근무형태_단순화'] = X_test.index.isin(c1_c2)

X_train['근무형태_단순화'] = X_train['근무형태_단순화'].apply(lambda x: '정규직' if x== True else '계약직')
X_test['근무형태_단순화'] = X_test['근무형태_단순화'].apply(lambda x: '정규직' if x== True else '계약직')

In [71]:
c1 = X_train.근무경력 >= 56
c2 = X_train['근무형태'] == '정규직'

c1_c2 = X_train.loc[c1&c2].index

c1t = X_test.근무경력 >= 56
c2t = X_test['근무형태'] == '정규직'

c1t_c2t = X_test.loc[c1t&c2t].index

X_train['베테랑정규직'] =  X_train.index.isin(c1_c2)
X_test['베테랑정규직'] =  X_test.index.isin(c1t_c2t)

## 33) 피처끼리의 합

In [72]:
X_train['근무경력+직무태그글자수범위별']= X_train['근무경력']+X_train['직무태그글자수범위별']
X_test['근무경력+직무태그글자수범위별']= X_test['근무경력']+X_test['직무태그글자수범위별']

X_train['근무경력+직무태그_직무수']= X_train['근무경력']+X_train['직무태그_직무수']
X_test['근무경력+직무태그_직무수']= X_test['근무경력']+X_test['직무태그_직무수']

X_train['근무경력+직종수']= X_train['근무경력']+X_train['직종수']
X_test['근무경력+직종수']= X_test['근무경력']+X_test['직종수']

X_train['근무경력+세부직종수']= X_train['근무경력']+X_train['세부직종수']
X_test['근무경력+세부직종수']= X_test['근무경력']+X_test['세부직종수']

X_train['근무경력+직종별근무경력합']= X_train['근무경력']+X_train['직종별근무경력합']
X_test['근무경력+직종별근무경력합']= X_test['근무경력']+X_test['직종별근무경력합']


## 34) 피처끼리의 곱

In [73]:
X_train['근무경력*직무태그글자수범위별']= X_train['근무경력']*X_train['직무태그글자수범위별']
X_test['근무경력*직무태그글자수범위별']= X_test['근무경력']*X_test['직무태그글자수범위별']

X_train['근무경력*직무태그_직무수']= X_train['근무경력']*X_train['직무태그_직무수']
X_test['근무경력*직무태그_직무수']= X_test['근무경력']*X_test['직무태그_직무수']

X_train['근무경력*직종수']= X_train['근무경력']*X_train['직종수']
X_test['근무경력*직종수']= X_test['근무경력']*X_test['직종수']

X_train['근무경력*세부직종수']= X_train['근무경력']*X_train['세부직종수']
X_test['근무경력*세부직종수']= X_test['근무경력']*X_test['세부직종수']

X_train['직무태그_직무수*직무태그글자수범위별']= X_train['직무태그_직무수']*X_train['직무태그글자수범위별']
X_test['직무태그_직무수*직무태그글자수범위별']= X_test['직무태그_직무수']*X_test['직무태그글자수범위별']

X_train['직종수*직무태그글자수범위별']= X_train['직종수']*X_train['직무태그글자수범위별']
X_test['직종수*직무태그글자수범위별']= X_test['직종수']*X_test['직무태그글자수범위별']

X_train['세부직종수*직무태그글자수범위별']= X_train['세부직종수']*X_train['직무태그글자수범위별']
X_test['세부직종수*직무태그글자수범위별']= X_test['세부직종수']*X_test['직무태그글자수범위별']

## 35) 마지막근무형태,근무지역,직무태그,출신대학별 근무경력, 대학성적

In [74]:
X_train=pd.merge(X_train, X_train.groupby('근무지역')['근무경력'].mean().reset_index().rename(columns={'근무경력':'근무지역별 근무경력mean'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['근무경력'].mean().reset_index().rename(columns={'근무경력':'근무지역별 근무경력mean'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['근무경력'].median().reset_index().rename(columns={'근무경력':'근무지역별 근무경력median'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['근무경력'].median().reset_index().rename(columns={'근무경력':'근무지역별 근무경력median'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['근무경력'].sum().reset_index().rename(columns={'근무경력':'근무지역별 근무경력sum'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['근무경력'].sum().reset_index().rename(columns={'근무경력':'근무지역별 근무경력sum'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['근무경력'].var().reset_index().rename(columns={'근무경력':'근무지역별 근무경력var'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['근무경력'].var().reset_index().rename(columns={'근무경력':'근무지역별 근무경력var'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['근무경력'].std().reset_index().rename(columns={'근무경력':'근무지역별 근무경력std'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['근무경력'].std().reset_index().rename(columns={'근무경력':'근무지역별 근무경력std'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['근무경력'].min().reset_index().rename(columns={'근무경력':'근무지역별 근무경력min'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['근무경력'].min().reset_index().rename(columns={'근무경력':'근무지역별 근무경력min'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['근무경력'].max().reset_index().rename(columns={'근무경력':'근무지역별 근무경력max'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['근무경력'].max().reset_index().rename(columns={'근무경력':'근무지역별 근무경력max'}), on = '근무지역',how='left')

In [75]:
X_train=pd.merge(X_train, X_train.groupby('근무지역')['대학성적'].mean().reset_index().rename(columns={'대학성적':'근무지역별 대학성적mean'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['대학성적'].mean().reset_index().rename(columns={'대학성적':'근무지역별 대학성적mean'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['대학성적'].median().reset_index().rename(columns={'대학성적':'근무지역별 대학성적median'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['대학성적'].median().reset_index().rename(columns={'대학성적':'근무지역별 대학성적median'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['대학성적'].sum().reset_index().rename(columns={'대학성적':'근무지역별 대학성적sum'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['대학성적'].sum().reset_index().rename(columns={'대학성적':'근무지역별 대학성적sum'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['대학성적'].var().reset_index().rename(columns={'대학성적':'근무지역별 대학성적var'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['대학성적'].var().reset_index().rename(columns={'대학성적':'근무지역별 대학성적var'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['대학성적'].std().reset_index().rename(columns={'대학성적':'근무지역별 대학성적std'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['대학성적'].std().reset_index().rename(columns={'대학성적':'근무지역별 대학성적std'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['대학성적'].min().reset_index().rename(columns={'대학성적':'근무지역별 대학성적min'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['대학성적'].min().reset_index().rename(columns={'대학성적':'근무지역별 대학성적min'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['대학성적'].max().reset_index().rename(columns={'대학성적':'근무지역별 대학성적max'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['대학성적'].max().reset_index().rename(columns={'대학성적':'근무지역별 대학성적max'}), on = '근무지역',how='left')

In [76]:
X_train=pd.merge(X_train, X_train.groupby('직무태그')['대학성적'].mean().reset_index().rename(columns={'대학성적':'직무태그별 대학성적mean'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['대학성적'].mean().reset_index().rename(columns={'대학성적':'직무태그별 대학성적mean'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['대학성적'].median().reset_index().rename(columns={'대학성적':'직무태그별 대학성적median'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['대학성적'].median().reset_index().rename(columns={'대학성적':'직무태그별 대학성적median'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['대학성적'].sum().reset_index().rename(columns={'대학성적':'직무태그별 대학성적sum'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['대학성적'].sum().reset_index().rename(columns={'대학성적':'직무태그별 대학성적sum'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['대학성적'].var().reset_index().rename(columns={'대학성적':'직무태그별 대학성적var'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['대학성적'].var().reset_index().rename(columns={'대학성적':'직무태그별 대학성적var'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['대학성적'].std().reset_index().rename(columns={'대학성적':'직무태그별 대학성적std'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['대학성적'].std().reset_index().rename(columns={'대학성적':'직무태그별 대학성적std'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['대학성적'].min().reset_index().rename(columns={'대학성적':'직무태그별 대학성적min'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['대학성적'].min().reset_index().rename(columns={'대학성적':'직무태그별 대학성적min'}), on = '직무태그',how='left')

X_train=pd.merge(X_train, X_train.groupby('직무태그')['대학성적'].max().reset_index().rename(columns={'대학성적':'직무태그별 대학성적max'}), on = '직무태그',how='left')
X_test=pd.merge(X_test, X_test.groupby('직무태그')['대학성적'].max().reset_index().rename(columns={'대학성적':'직무태그별 대학성적max'}), on = '직무태그',how='left')

## 36)근무경력_세분화

In [77]:
def seg_worklength(x):
    if 0.0 <= x < 24.0:
        x = '0to2'
    elif 24 <= x < 48:
        x = '2to4'
    elif 48 <= x < 72: 
        x = '4to6'
    elif 72 <= x < 96:
        x = '6to8'
    elif 96 <= x < 120:
        x = '8to10'
    elif 120 <= x:
        x = 'over10'
    return x
X_train['근무경력_세분화'] = X_train['근무경력'].apply(seg_worklength)
X_test['근무경력_세분화'] =X_test['근무경력'].apply(seg_worklength)

## 37) 해외근무지역

In [78]:
korea = ['서울','부산', '경기', '전국', '기타', '전북', '충남', '광주', '인천', '대전', '경남','충북','울산', '경북', '강원','전남'
        ,'대구','제주','광주']
foreign = ['홍콩','미국','해외','중국','캐나다','인도네시아','일본','싱가포르','필리핀','대만','말레이시아','인도네시아','러시아','인도'
          ,'프랑스','필리핀','말레이시아','방글라데시']
world = korea + foreign

f_f = X_train.근무지역.str.split(',',expand=True)[0].isin(foreign)
s_f = X_train.근무지역.str.split(',',expand=True)[1].isin(foreign)
t_f = X_train.근무지역.str.split(',',expand=True)[2].isin(foreign)

c = (f_f | s_f | t_f)

c_index = X_train.loc[c,'근무지역'].index

X_train['해외근무지역'] =  (X_train.index).isin(c_index)

f_f = X_test.근무지역.str.split(',',expand=True)[0].isin(foreign)
s_f = X_test.근무지역.str.split(',',expand=True)[1].isin(foreign)
t_f = X_test.근무지역.str.split(',',expand=True)[2].isin(foreign)

c = (f_f | s_f | t_f)

ct_index = X_test.loc[c,'근무지역'].index

X_test['해외근무지역'] =  (X_test.index).isin(ct_index)

X_train['해외근무지역'] = X_train['해외근무지역'].apply(lambda x: '유' if x== True else '무')
X_test['해외근무지역'] = X_test['해외근무지역'].apply(lambda x: '유' if x== True else '무')

## 38) new_ 근무지역

In [79]:
def list_for_new_df(values,lists):
    result = np.zeros_like(lists,dtype=int)
    
    for value in values: 
        try:
            result_index = lists.index(value)
        except:
            continue
        result[result_index] = 1
    
    return result

In [80]:
# 근무지역 겹치는거 많음, 새롭게 만들어야함
def region(x):
    if x in ['서울']:
        x = '수도권_서울'
    elif x in ['경기']:
        x = '수도권_경기'
    elif x in ['인천']:
        x = '수도권_인천'
    elif x in ['강원']:
        x = '강원'
    elif x in ['경남','경북','대구','부산','울산']:
        x = '영남'
    elif x in ['광주','전남','전북']:
        x = '호남'
    elif x in ['충남','충북']:
        x = '호서'
    elif x in ['제주']:
        x = '제주'
    elif x in ['대만','러시아','말레이시아','방글라데시','싱가포르','인도','인도네시아','일본','중국','필리핀','홍콩']:
        x = '해외_유라시아'
    elif x in ['미국','캐나다']:
        x = '해외_아메리카'
    elif x in ['프랑스','해외']:
        x = '해외_기타'
    elif x == '':
        x = ''
    else: 
        x = '기타'
    return x

def new_region(x):
    x = set(sorted(map(region,x.split(','))))
    x.discard('')
    return tuple(x)


X_train['new_근무지역'] = X_train['근무지역'].apply(new_region)
X_test['new_근무지역'] = X_test['근무지역'].apply(new_region)

## 39) 근무지역_경험횟수

In [81]:
X_train['근무지역_경험횟수'] = X_train['new_근무지역'].apply(lambda x: len(x))
X_test['근무지역_경험횟수'] = X_test['new_근무지역'].apply(lambda x: len(x))

## 40) 근무지역 관련 피처 생성

In [82]:
region = ['수도권_서울','수도권_경기','수도권_인천','강원','호남','호서','영남','제주',
          '해외_유라시아','해외_아메리카','해외_기타','기타']

X_train_region_list = []
for values in X_train['new_근무지역'].values:
    X_train_region = list_for_new_df(values,region)
    X_train_region_list.append(X_train_region)

X_test_region_list = []
for values in X_test['new_근무지역'].values:
    X_test_region = list_for_new_df(values,region)
    X_test_region_list.append(X_test_region)
    
    
columns = ['근무지역_수도권_서울','근무지역_수도권_경기','근무지역_수도권_인천','근무지역_강원','근무지역_호남',
           '근무지역_호서','근무지역_영남','근무지역_제주',
           '근무지역_해외_유라시아','근무지역_해외_아메리카','근무지역_해외_기타','근무지역_기타']

X_train_region_df = pd.DataFrame((np.vstack(X_train_region_list)),columns = columns)
X_test_region_df = pd.DataFrame((np.vstack(X_test_region_list)),columns = columns)

X_train = pd.concat((X_train,X_train_region_df),axis=1)
X_test = pd.concat((X_test,X_test_region_df),axis=1)

del X_train_region_df
del X_test_region_df

## 41) 수도권 대학교

In [83]:
def distance_to_seoul(x):
    closer_to_seoul = ['성균관대학교','한성대학교', '세종대학교', '성신여자대학교','연세대학교', '동덕여자대학교',
                       '성공회대학교','중앙대학교','이화여자대학교','서울여자대학교','수원대학교','인천대학교',
                       '경기대학교','성결대학교','용인대학교','협성대학교','한국산업기술대학교','서울신학대학교','서울과학기술대학교']
    if x in closer_to_seoul:
        x = 1
    else:
        x = 0
    return x


X_train['수도권대학교'] = X_train['출신대학'].apply(distance_to_seoul)
X_test['수도권대학교'] = X_test['출신대학'].apply(distance_to_seoul)

In [84]:
X_train['대학지역']='기타'
X_test['대학지역']='기타'

X_train.loc[(X_train['출신대학']=='성균관대학교')|(X_train['출신대학']=='중앙대학교')
           |(X_train['출신대학']=='세종대학교')|(X_train['출신대학']=='연세대학교')|(X_train['출신대학']=='한성대학교')
           |(X_train['출신대학']=='성신여자대학교')|(X_train['출신대학']=='동덕여자대학교')|(X_train['출신대학']=='이화여자대학교')
           |(X_train['출신대학']=='한림대학교')|(X_train['출신대학']=='서울여자대학교')|(X_train['출신대학']=='서울과학기술대학교')
           |(X_train['출신대학']=='서울신학대학교')|(X_train['출신대학']=='성공회대학교'),'대학지역']='서울'

X_train.loc[(X_train['출신대학']=='동아대학교')|(X_train['출신대학']=='동의대학교')
           |(X_train['출신대학']=='신라대학교')|(X_train['출신대학']=='동의대학교')|(X_train['출신대학']=='경주대학교')
          |(X_train['출신대학']=='부산디지털대학교')|(X_train['출신대학']=='동서대학교'),'대학지역']='영남'

X_train.loc[(X_train['출신대학']=='수원대학교')
           |(X_train['출신대학']=='경기대학교')|(X_train['출신대학']=='호원대학교')|(X_train['출신대학']=='용인대학교')
          |(X_train['출신대학']=='협성대학교')|(X_train['출신대학']=='한국산업기술대학교')
          |(X_train['출신대학']=='인천대학교')|(X_train['출신대학']=='성결대학교')|(X_train['출신대학']=='한세대학교'),'대학지역']='경기'

X_train.loc[(X_train['출신대학']=='한밭대학교')|(X_train['출신대학']=='세명대학교')
           |(X_train['출신대학']=='선문대학교')|(X_train['출신대학']=='서원대학교')
          ,'대학지역']='호서'

X_train.loc[(X_train['출신대학']=='전주대학교')|(X_train['출신대학']=='목포대학교')
           |(X_train['출신대학']=='군산대학교')|(X_train['출신대학']=='호남대학교')|(X_train['출신대학']=='광주대학교')
          |(X_train['출신대학']=='한일장신대학교')|(X_train['출신대학']=='송원대학교'),'대학지역']='호남'

X_train.loc[(X_train['출신대학']=='제주대학교'),'대학지역']='제주'

X_train.loc[(X_train['출신대학']=='상지대학교'),'대학지역']='강원'

#X_test
X_test.loc[(X_test['출신대학']=='성균관대학교')|(X_test['출신대학']=='중앙대학교')
           |(X_test['출신대학']=='세종대학교')|(X_test['출신대학']=='연세대학교')|(X_test['출신대학']=='한성대학교')
           |(X_test['출신대학']=='성신여자대학교')|(X_test['출신대학']=='동덕여자대학교')|(X_test['출신대학']=='이화여자대학교')
           |(X_test['출신대학']=='한림대학교')|(X_test['출신대학']=='서울여자대학교')|(X_test['출신대학']=='서울과학기술대학교')
           |(X_test['출신대학']=='서울신학대학교')|(X_test['출신대학']=='성공회대학교'),'대학지역']='서울'

X_test.loc[(X_test['출신대학']=='동아대학교')|(X_test['출신대학']=='동의대학교')
           |(X_test['출신대학']=='신라대학교')|(X_test['출신대학']=='동의대학교')|(X_test['출신대학']=='경주대학교')
          |(X_test['출신대학']=='부산디지털대학교')|(X_test['출신대학']=='동서대학교'),'대학지역']='영남'

X_test.loc[(X_test['출신대학']=='수원대학교')
           |(X_test['출신대학']=='경기대학교')|(X_test['출신대학']=='호원대학교')|(X_test['출신대학']=='용인대학교')
          |(X_test['출신대학']=='협성대학교')|(X_test['출신대학']=='한국산업기술대학교')
          |(X_test['출신대학']=='인천대학교')|(X_test['출신대학']=='성결대학교')|(X_test['출신대학']=='한세대학교'),'대학지역']='경기'

X_test.loc[(X_test['출신대학']=='한밭대학교')|(X_test['출신대학']=='세명대학교')
           |(X_test['출신대학']=='선문대학교')|(X_test['출신대학']=='서원대학교')
          ,'대학지역']='호서'

X_test.loc[(X_test['출신대학']=='전주대학교')|(X_test['출신대학']=='목포대학교')
           |(X_test['출신대학']=='군산대학교')|(X_test['출신대학']=='호남대학교')|(X_test['출신대학']=='광주대학교')
          |(X_test['출신대학']=='한일장신대학교')|(X_test['출신대학']=='송원대학교'),'대학지역']='호남'

X_test.loc[(X_test['출신대학']=='제주대학교'),'대학지역']='제주'

X_test.loc[(X_test['출신대학']=='상지대학교'),'대학지역']='강원'

## 42)연고직

In [85]:
X_train['연고지'] = 0
X_train.loc[(X_train['대학지역']=='서울')&(X_train['근무지역_수도권_서울']==1),'연고지']=1
X_train.loc[(X_train['대학지역']=='경기')&(X_train['근무지역_수도권_경기']==1),'연고지']=1
X_train.loc[(X_train['대학지역']=='영남')&(X_train['근무지역_영남']==1),'연고지']=1
X_train.loc[(X_train['대학지역']=='호서')&(X_train['근무지역_호서']==1),'연고지']=1
X_train.loc[(X_train['대학지역']=='호남')&(X_train['근무지역_호남']==1),'연고지']=1
X_train.loc[(X_train['대학지역']=='강원')&(X_train['근무지역_강원']==1),'연고지']=1
X_train.loc[(X_train['대학지역']=='제주')&(X_train['근무지역_제주']==1),'연고지']=1


X_test['연고지'] = 0
X_test.loc[(X_test['대학지역']=='서울')&(X_test['근무지역_수도권_서울']==1),'연고지']=1
X_test.loc[(X_train['대학지역']=='경기')&(X_test['근무지역_수도권_경기']==1),'연고지']=1
X_test.loc[(X_train['대학지역']=='영남')&(X_test['근무지역_영남']==1),'연고지']=1
X_test.loc[(X_train['대학지역']=='호서')&(X_test['근무지역_호서']==1),'연고지']=1
X_test.loc[(X_train['대학지역']=='호남')&(X_test['근무지역_호남']==1),'연고지']=1
X_test.loc[(X_train['대학지역']=='강원')&(X_test['근무지역_강원']==1),'연고지']=1
X_test.loc[(X_train['대학지역']=='제주')&(X_test['근무지역_제주']==1),'연고지']=1

## 43) 명문대

In [86]:
X_train.loc[(X_train['출신대학'].str.contains('성균관대학교|세종대학교|성신여자대학교|연세대학교|동덕여자대학교|중앙대학교|이화여자대학교|서울과학기술대학교|서울여자대학교')),'명문대']=1
X_train['명문대']=X_train['명문대'].fillna(0)

X_test.loc[(X_test['출신대학'].str.contains('성균관대학교|세종대학교|성신여자대학교|연세대학교|동덕여자대학교|중앙대학교|이화여자대학교|서울과학기술대학교|서울여자대학교')),'명문대']=1
X_test['명문대']=X_test['명문대'].fillna(0)

In [87]:
X = pd.concat([X_train, y_train], axis = 1)

In [88]:
def merge_region_worklength(data,on):
    right = X.groupby(on)[['근무경력']].mean()
    right.loc[0] = 0.0
    right.columns = [f"{on}_경력평균"]
    result = pd.merge(data,right,on=on,how='left')
    return result

on_list = ['근무지역_수도권_서울','근무지역_수도권_경기','근무지역_수도권_인천','근무지역_강원','근무지역_호남',
           '근무지역_호서','근무지역_영남','근무지역_제주',
           '근무지역_해외_유라시아','근무지역_해외_아메리카','근무지역_해외_기타','근무지역_기타']


#X_train merge
for on in on_list:
    X_train = merge_region_worklength(X_train,on)
# X_test merge
for on in on_list:
    X_test = merge_region_worklength(X_test,on)

In [89]:
region_worklength_mean_list = ['근무지역_수도권_서울_경력평균', '근무지역_수도권_경기_경력평균', '근무지역_수도권_인천_경력평균',
       '근무지역_강원_경력평균', '근무지역_호남_경력평균', '근무지역_호서_경력평균', '근무지역_영남_경력평균',
       '근무지역_제주_경력평균', '근무지역_해외_유라시아_경력평균', '근무지역_해외_아메리카_경력평균',
       '근무지역_해외_기타_경력평균', '근무지역_기타_경력평균']

In [90]:
X_train['근무지역_개별_경력평균'] = X_train[region_worklength_mean_list].sum(axis=1)/X_train['근무지역_경험횟수']
X_test['근무지역_개별_경력평균'] = X_test[region_worklength_mean_list].sum(axis=1)/X_test['근무지역_경험횟수']

## 44) 근무지역1별 대학성적, 근무경력

In [91]:
X_train=pd.merge(X_train, X_train.groupby('근무지역1')['대학성적'].mean().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적mean'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['대학성적'].mean().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적mean'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['대학성적'].median().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적median'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['대학성적'].median().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적median'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['대학성적'].sum().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적sum'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['대학성적'].sum().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적sum'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['대학성적'].var().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적var'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['대학성적'].var().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적var'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['대학성적'].std().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적std'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['대학성적'].std().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적std'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['대학성적'].min().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적min'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['대학성적'].min().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적min'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['대학성적'].max().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적max'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['대학성적'].max().reset_index().rename(columns={'대학성적':'근무지역1별 대학성적max'}), on = '근무지역1',how='left')

In [92]:
X_train=pd.merge(X_train, X_train.groupby('근무지역1')['근무경력'].mean().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력mean'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['근무경력'].mean().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력mean'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['근무경력'].sum().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력sum'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['근무경력'].sum().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력sum'}), on = '근무지역1',how='left')


X_train=pd.merge(X_train, X_train.groupby('근무지역1')['근무경력'].median().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력median'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['근무경력'].median().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력median'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['근무경력'].var().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력var'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['근무경력'].var().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력var'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['근무경력'].std().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력std'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['근무경력'].std().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력std'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['근무경력'].min().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력min'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['근무경력'].min().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력min'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['근무경력'].max().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력max'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['근무경력'].max().reset_index().rename(columns={'근무경력':'근무지역1별 근무경력max'}), on = '근무지역1',how='left')

## 45) 숙련도별 대학성적, 근무경력

In [93]:
X_train=pd.merge(X_train, X_train.groupby('숙련도')['대학성적'].mean().reset_index().rename(columns={'대학성적':'숙련도별 대학성적mean'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['대학성적'].mean().reset_index().rename(columns={'대학성적':'숙련도별 대학성적mean'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['대학성적'].median().reset_index().rename(columns={'대학성적':'숙련도별 대학성적median'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['대학성적'].median().reset_index().rename(columns={'대학성적':'숙련도별 대학성적median'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['대학성적'].sum().reset_index().rename(columns={'대학성적':'숙련도별 대학성적sum'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['대학성적'].sum().reset_index().rename(columns={'대학성적':'숙련도별 대학성적sum'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['대학성적'].var().reset_index().rename(columns={'대학성적':'숙련도별 대학성적var'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['대학성적'].var().reset_index().rename(columns={'대학성적':'숙련도별 대학성적var'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['대학성적'].std().reset_index().rename(columns={'대학성적':'숙련도별 대학성적std'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['대학성적'].std().reset_index().rename(columns={'대학성적':'숙련도별 대학성적std'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['대학성적'].min().reset_index().rename(columns={'대학성적':'숙련도별 대학성적min'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['대학성적'].min().reset_index().rename(columns={'대학성적':'숙련도별 대학성적min'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['대학성적'].max().reset_index().rename(columns={'대학성적':'숙련도별 대학성적max'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['대학성적'].max().reset_index().rename(columns={'대학성적':'숙련도별 대학성적max'}), on = '숙련도',how='left')

In [94]:
X_train=pd.merge(X_train, X_train.groupby('숙련도')['근무경력'].mean().reset_index().rename(columns={'근무경력':'숙련도별 근무경력mean'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['근무경력'].mean().reset_index().rename(columns={'근무경력':'숙련도별 근무경력mean'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['근무경력'].sum().reset_index().rename(columns={'근무경력':'숙련도별 근무경력sum'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['근무경력'].sum().reset_index().rename(columns={'근무경력':'숙련도별 근무경력sum'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['근무경력'].median().reset_index().rename(columns={'근무경력':'숙련도별 근무경력median'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['근무경력'].median().reset_index().rename(columns={'근무경력':'숙련도별 근무경력median'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['근무경력'].var().reset_index().rename(columns={'근무경력':'숙련도별 근무경력var'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['근무경력'].var().reset_index().rename(columns={'근무경력':'숙련도별 근무경력var'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['근무경력'].std().reset_index().rename(columns={'근무경력':'숙련도별 근무경력std'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['근무경력'].std().reset_index().rename(columns={'근무경력':'숙련도별 근무경력std'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['근무경력'].min().reset_index().rename(columns={'근무경력':'숙련도별 근무경력min'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['근무경력'].min().reset_index().rename(columns={'근무경력':'숙련도별 근무경력min'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['근무경력'].max().reset_index().rename(columns={'근무경력':'숙련도별 근무경력max'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['근무경력'].max().reset_index().rename(columns={'근무경력':'숙련도별 근무경력max'}), on = '숙련도',how='left')

## 46) 숙련도별 대학성적 , 근무경력,출신대학수치

In [95]:
X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['대학성적'].mean().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적mean'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['대학성적'].mean().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적mean'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['대학성적'].median().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적median'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['대학성적'].median().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적median'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['대학성적'].sum().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적sum'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['대학성적'].sum().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적sum'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['대학성적'].var().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적var'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['대학성적'].var().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적var'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['대학성적'].std().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적std'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['대학성적'].std().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적std'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['대학성적'].min().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적min'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['대학성적'].min().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적min'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['대학성적'].max().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적max'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['대학성적'].max().reset_index().rename(columns={'대학성적':'근무경력_세분화별 대학성적max'}), on = '근무경력_세분화',how='left')

In [96]:
X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['근무경력'].mean().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력mean'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['근무경력'].mean().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력mean'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['근무경력'].sum().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력sum'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['근무경력'].sum().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력sum'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['근무경력'].median().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력median'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['근무경력'].median().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력median'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['근무경력'].var().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력var'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['근무경력'].var().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력var'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['근무경력'].std().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력std'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['근무경력'].std().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력std'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['근무경력'].min().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력min'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['근무경력'].min().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력min'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['근무경력'].max().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력max'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['근무경력'].max().reset_index().rename(columns={'근무경력':'근무경력_세분화별 근무경력max'}), on = '근무경력_세분화',how='left')

In [97]:
X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치mean'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치mean'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치median'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치median'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치sum'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치sum'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치var'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치var'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치std'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치std'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치min'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치min'}), on = '근무경력_세분화',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무경력_세분화')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치max'}), on = '근무경력_세분화',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무경력_세분화')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'근무경력_세분화별 출신대학수치max'}), on = '근무경력_세분화',how='left')


## 47) ㅇㅇ별 출신대학수치

In [98]:
X_train=pd.merge(X_train, X_train.groupby('출신대학')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치mean'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치mean'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치median'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치median'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치sum'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치sum'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치var'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치var'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치std'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치std'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치min'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치min'}), on = '출신대학',how='left')

X_train=pd.merge(X_train, X_train.groupby('출신대학')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치max'}), on = '출신대학',how='left')
X_test=pd.merge(X_test, X_test.groupby('출신대학')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'출신대학별 출신대학수치max'}), on = '출신대학',how='left')

In [99]:
X_train=pd.merge(X_train, X_train.groupby('근무지역')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치mean'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치mean'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치median'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치median'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치sum'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치sum'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치var'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치var'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치std'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치std'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치min'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치min'}), on = '근무지역',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치max'}), on = '근무지역',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'근무지역별 출신대학수치max'}), on = '근무지역',how='left')

In [100]:
X_train=pd.merge(X_train, X_train.groupby('근무지역1')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치mean'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치mean'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치median'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치median'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치sum'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치sum'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치var'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치var'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치std'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치std'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치min'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치min'}), on = '근무지역1',how='left')

X_train=pd.merge(X_train, X_train.groupby('근무지역1')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치max'}), on = '근무지역1',how='left')
X_test=pd.merge(X_test, X_test.groupby('근무지역1')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'근무지역1별 출신대학수치max'}), on = '근무지역1',how='left')

In [101]:
X_train=pd.merge(X_train, X_train.groupby('숙련도')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치mean'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['출신대학수치'].mean().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치mean'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치median'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['출신대학수치'].median().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치median'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치sum'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['출신대학수치'].sum().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치sum'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치var'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['출신대학수치'].var().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치var'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치std'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['출신대학수치'].std().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치std'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치min'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['출신대학수치'].min().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치min'}), on = '숙련도',how='left')

X_train=pd.merge(X_train, X_train.groupby('숙련도')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치max'}), on = '숙련도',how='left')
X_test=pd.merge(X_test, X_test.groupby('숙련도')['출신대학수치'].max().reset_index().rename(columns={'출신대학수치':'숙련도별 출신대학수치max'}), on = '숙련도',how='left')

## 48) 출신대학별 근무경력mean과 근무경력의 곱, 합, 뺄셈

In [102]:
X_train['출신대학별 근무경력mean*근무경력']=X_train['출신대학별 근무경력mean']*X_train['근무경력']
X_test['출신대학별 근무경력mean*근무경력']=X_test['출신대학별 근무경력mean']*X_test['근무경력']

X_train['출신대학별 근무경력mean+근무경력']=X_train['출신대학별 근무경력mean']+X_train['근무경력']
X_test['출신대학별 근무경력mean+근무경력']=X_test['출신대학별 근무경력mean']+X_test['근무경력']

X_train['출신대학별 근무경력mean-근무경력']=X_train['출신대학별 근무경력mean']-X_train['근무경력']
X_test['출신대학별 근무경력mean-근무경력']=X_test['출신대학별 근무경력mean']-X_test['근무경력']

## 49) 오타만 있는 피처 새로 생성

In [103]:
ota_major = ['1', '졸업','전공','토&#47791공학', 'e-businees학', '산', '영','멀티미디어학부(인터넷응요)','InternationalSummerProgram',
      '어학연수','ㅇ','ㅇㅇ', '조리외식졍영','사','철철','토옥','철','새왈과학부','외식사업','영','문화컨테츠',
      '사체','eh', '전공', 'dddddd', '1', '3.03', 'ghj','주거솬경학','회힉공학','Theatre','경역학','체욱학','문화제보존학','건충공학'
     ,'기게공학','&#49887;품공학','law','수원대학교3학년편입후졸업','ㅁㅇㄻㅈㄷㄹ','physics','없음','MIS','관공통역','수원대학교',
     'SocialHumanMovementandperformance','관좡학','library&info.science','환격공학','CommunicationConsulting','핼정학','마이컴',
     '웨이트트레이닝','이공계열중퇴','화확공학']

bad_boy = []
for i in X_train['대학전공']:
    if i in ota_major:
        bad_boy.append(0)
    else:
        bad_boy.append(1)

X_train['bad_boy'] = bad_boy      

bad_boy2 = []
for i in X_test['대학전공']:
    if i in ota_major:
        bad_boy2.append(0)
    else:
        bad_boy2.append(1)
        
X_test['bad_boy'] = bad_boy2

In [104]:
ota_tag = ['시원하게따사롭게','7277','영업,영없관리','없음','소리없이끈기있는...','생산관리분야에관한경력은없지만기회가있으면근무하고싶습니다',
        '8483','2172','aaaa','패기.열정이있는젊은이!나를선택해주십시요!','ㅇㅇ','12ㅂㅈ', 'ㅁㅁ', 'ㄷ','회사의발전이곧나의발전이라생각하며열심히일하겠습니다.',
        '여러모임에서의회장을역임한경험이큰발전의계기가될것입니다.','LGDisplay신공장기획및국내영업경력이력서입니다','자금조달,운용,집행및총무,인사업무경력자입니다.',
         '능력있는전자구매자입니다.','열심히노력하는RF엔지니어입니다','서인호이력서입니다','준비된자세한상민입니다','이제3년차에접어든재무요원입니다...','난감동적인사람이다',
         '포기는김장철에나쓰는말이다.','불가능그것은도전이라는이름앞에무너지는것이다.','TOEIC940,무역전공,신입,해외영업을하고싶습니다.','무','.','p','웹','일','1','법','C','??',
        '금웅','금용','급여','성찰','성실','조선','중국','지원','희망','현지','429','멋쟁이','새출발','신동영','심응재','써비스','윤재임','jw134','alsl1'
        '조주현','준비중','최영연','최희주','1052','1201','1878','7770','test','pc가능','박미선','보상과','중국통','변화무쌍','트레이너','11111','준비된비서',
         '#NAME?','kj0121','cccccc','gun3733','70013867','bong9411','농산물Buyer','영업밖에모르는놈','21세기새로운도전','능력있는영업지원자','무엇이든할수있어요',
        '소리없이끈기있는..','회계담당자...경리','회계의모든것은나에게로','사무,총부,인사....','회계,세무분야성실한인재',',영어,일본어,번역,통역','1금융권에서10년이상근무',
         '성실하고능력있는웹디자이너','★언제나처음과같은마음으로★','김세미의해외영업직지원이력서','해외영업분야인재를찾고계십니까',"반도체,공정eng'r,Engineer,진공",'최선을다한나',
         '제가필요로하는곳.','일러스트,,#53245;,포토,#49406;,드림위버,기획력',',','0','?','s','ㅇ','영','!!','6σ','11','임대호',',,,','207','310','간다잉','금융外','이진혁',
         '3240','4831','9336','ho30','비전21','나의이력서','bar,까페','내가해결하라','최영철이력서','yesu153','뛰는영업관리자','leejs718','후회없는삶을살자','강력한서비스마인드','#49406;매니저',
         ]


bad_girl = []
for i in X_train['직무태그']:
    if i in ota_tag:
        bad_girl.append(0)
    else:
        bad_girl.append(1)

X_train['bad_girl'] = bad_girl      

bad_girl2 = []
for i in X_test['직무태그']:
    if i in ota_tag:
        bad_girl2.append(0)
    else:
        bad_girl2.append(1)
        
X_test['bad_girl'] = bad_girl2  

# 데이터 내보내기

In [105]:
X_train.to_csv('X_train_피처추가1124지협지0357.csv',encoding='cp949', index=False)

In [106]:
X_test.to_csv('X_test_피처추가1124지협지0357.csv',encoding='cp949', index=False)